In [ ]:
# 벡터저장소가 이미 있는 상황
from dotenv import load_dotenv  # api-key

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# 1. 벡터 스토어 가져오기
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
persist_directory = "../07_vectorstore/samsung_2025_db"
collection_name = "samsung2025"

vectorstore = Chroma(
    persist_directory=persist_directory,
    collection_name=collection_name,
    embedding_function=embedding
)

In [ ]:
# 2. retriever 만들기
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 30}
)

In [ ]:
# 3. reranker 만들기
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

hf_ce = HuggingFaceCrossEncoder(
    model_name = "cross-encoder/ms-marco-MiniLM-L6-v2",
    model_kwargs = {
        "device" : "cuda",
        "max_length" : 512,
    }
)
reranker = CrossEncoderReranker(
    model = hf_ce,
    top_n = 10
)

In [ ]:
# 4. retriever -> reranker
from langchain.retrievers import ContextualCompressionRetriever

com_retriever = ContextualCompressionRetriever(
    base_retriever = retriever,
    base_compressor = reranker
)

In [ ]:
from langchain_community.document_transformers import LongContextReorder

# 5. reorder(순서 정리)
reorder = LongContextReorder()

In [ ]:
# 6. 검색 결과 문서 합치는 함수 생성
def format_docs(docs):
    result = []
    for item in docs:
        result.append(item.page_content)
    return "\n\n---\n\n".join(result)

### 2. 기본 체안 만들기

In [ ]:
# 1. 프롬프트 설정
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. 
     
     [컨텍스트]
     {context}
     
     """),
    ("human", "{question}")
])

# 2. 모델 설정
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature=0
)

# 3. outputparser 
outputparser = StrOutputParser()

# 4. 체인 설정
chain = rag_prompt | model | outputparser
chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. \n\n     [컨텍스트]\n     {context}\n\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021E835A2990>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021E81F09DD0>, root_client=<openai.OpenAI object at 0x0000021E833CB790>, root_async_client=<openai.AsyncOpenAI object at 0x0000021E82D6A010>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser()

### 3. 통합 체인 만들기

In [ ]:
rag_chain  = (
    {"docs" : RunnableLambda(lambda x: com_retriever.invoke(x["question"])),
     "question" : RunnablePassthrough()
     }
     | RunnableLambda(lambda x : {
         "context" : format_docs(reorder.transform_documents(x["docs"])),
         "question" : x['question']
     })
     | chain
)

rag_chain.invoke({
    "question" : "삼성의 미래 계획은 어떻게 되나요?"
})

"삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래 기술 리더십으로 재도약의 기반을 다지고, 새로운 영역에서 미래 성장동력을 확보해 나갈 계획입니다. 또한, 2025년에는 '삼성 청년SW·AI아카데미' 교육 기회를 마이스터고 졸업생까지 확대하고, '삼성 희망디딤돌' 인천센터를 추가 설립하여 더 많은 청년을 지원할 예정입니다. 환경 분야에서는 2050년 탄소중립 달성을 목표로 온실가스 감축과 재생에너지 전환 등 중장기 전략을 추진하고 있습니다."

In [ ]:
rag_chain

{
  docs: RunnableLambda(lambda x: com_retriever.invoke(x['question'])),
  question: RunnablePassthrough()
}
| RunnableLambda(...)
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. \n\n     [컨텍스트]\n     {context}\n\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021E81BF1E50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021E81BF0250>, root_client=<openai.OpenAI object at 0x0000021E81BF1610>, root_async_client=<openai.AsyncOpenAI object at 0x0000021E82196050>, model_name='gpt-4.1-mi

### 4. multi_input chain

In [ ]:
com_retriever_chain = RunnableLambda(lambda x: x['question']) | com_retriever | format_docs

In [ ]:
rag_chain = (
    {
        "context" :  com_retriever_chain,
        "question" : RunnablePassthrough()
    }
    | chain
)
rag_chain.invoke({
    "question" : "삼성의 미래 계획은 어떻게 되나요?"
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래 기술 리더십으로 재도약의 기반을 다지고, 새로운 영역에서 미래 성장동력을 확보해 나갈 계획입니다. 또한, 지속가능한 성장 기반 마련을 위해 이해관계자의 의견에 귀 기울이며 지속적으로 노력할 예정입니다.'

In [ ]:
# 1. 프롬프트 설정
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. 
     
     [컨텍스트]
     {context}
     
     """),
    ("human", "{pro} 스타일에 맞게 {question}에 대답해라")
])

# 2. 모델 설정
MODEL_NAME =  "gpt-4.1-mini"
temperature = 0

model = ChatOpenAI(
    model = MODEL_NAME,
    temperature = temperature
)

# 3. outputparser 
outputparser = StrOutputParser()

# 4. 체인 설정
chain = rag_prompt | model | outputparser
chain

ChatPromptTemplate(input_variables=['context', 'pro', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. \n\n     [컨텍스트]\n     {context}\n\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pro', 'question'], input_types={}, partial_variables={}, template='{pro} 스타일에 맞게 {question}에 대답해라'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021E83594310>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021E835969D0>, root_client=<openai.OpenAI object at 0x0000021E6FDC5050>, root_async_client=<openai.AsyncOpenAI object at 0x0000021E835966D0>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
|

In [ ]:
# 1. 프롬프트 설정
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. 
     
     [컨텍스트]
     {context}
     
     """),
    ("human", "{pro} 스타일에 맞게 {question}에 대답해라")
])

# 2. 모델 설정
MODEL_NAME =  "gpt-4.1-mini"
temperature = 0

model = ChatOpenAI(
    model = MODEL_NAME,
    temperature = temperature
)

# 3. outputparser 
outputparser = StrOutputParser()

# 4. 체인 설정
chain = rag_prompt | model | outputparser

rag_chain = (
    {
        "context" :  RunnableLambda(lambda x: x['question']) | com_retriever | format_docs,
        "question" : RunnablePassthrough(),
        "pro" : RunnableLambda(lambda x: x['pro'])
    }
    | chain
)

# 실행 코드
rag_chain.invoke({
    "question" : "삼성의 미래 계획은 어떻게 되나요?",
    "pro" : "냥냥체"
})

'삼성은 인재와 기술을 바탕으로 최고의 제품과 서비스를 만들어 인류사회에 공헌할 계획이냥! 기술 리더십으로 재도약하고 새로운 영역에서 미래 성장동력을 확보할 거라냥! 지속가능한 성장 기반 마련에도 최선을 다할 예정이냥!'